In [ ]:
import os
import re
import time
import json
import random
from pprint import pprint
from pathlib import Path
from collections import defaultdict
from functools import partial
from tqdm.notebook import tqdm

from urllib.parse import urlencode, urlparse, urlunparse, parse_qs, unquote

import pandas as pd
import numpy as np

from omegaconf import OmegaConf

from transformations.id import IDRules

import ipyplot as iplt
import matplotlib.pyplot as plt

pd.options.display.max_columns = 150
pd.options.display.max_rows = 100

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# df = dfs["choice"]
# for col in df.columns:
#     print("-" * 100)
#     print(col)
#     print("-" * 100)
#     print(df[col].value_counts().head(20))

In [ ]:
def insert(row, merchant, col):
    x = row[col]
    _id = row["id"]
    if not pd.isna(x):
        if type(x) == float or (type(x) == str and x.isdigit()):
            x = int(x)
        if type(int) == int and x == 0:
            return
        if type(x) == str and x.strip() == "":
            return
        collisions[x].append({"merchant": merchant, "col": col, "id": _id})


collisions = defaultdict(list)
for merchant, df in tqdm(dfs.items()):
    print(merchant)
    for col in conf[merchant].get("code", {}):
        if col in df.columns:
            df.apply(partial(insert, merchant=merchant, col=col), axis=1)
        # else:
        #     print(f"{col} not in {merchant}")
len(collisions)


In [ ]:
df = pd.concat(dfs.values(), axis=0, ignore_index=True)
df.set_index("id", inplace=True)

In [ ]:
pd.Series(list(map(lambda x: len(set({i["merchant"] for i in x})), collisions.values()))).value_counts()

In [ ]:
codes = set()
for code, match in collisions.items():
    if len(set([i["merchant"] for i in match])) > 1:
        codes.add(code)
codes = list(codes)
len(codes)

In [ ]:
code = random.choice(codes)
print(code)
print(len(str(code)))
pprint(collisions[code], sort_dicts=False)
frame = df.loc[[x["id"] for x in collisions[code]], ["merchant", "brand", "product_name", "image_url", "gtin", "style"]].drop_duplicates()
iplt.plot_images(frame.image_url.values, show_url=False, img_width=200)
frame

In [ ]:
subset = set()
cols = set()
for code in codes:
    for match in collisions[code]:
        # if match["merchant"] == "very" and match["col"] == "merchant_product_id":
        if match["merchant"] == "vrnts":
            subset.add(code)
            cols.add(match["col"])
subset = list(subset)
print(len(subset))
print(cols)

In [ ]:
code = random.choice(subset)
print(code)
print(len(str(code)))
pprint(collisions[code], sort_dicts=False)
frame = df.loc[[x["id"] for x in collisions[code]], ["merchant", "brand", "product_name", "image_url", "gtin", "style"]].drop_duplicates()
iplt.plot_images(frame.image_url.values, show_url=False, img_width=200)
frame